In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql

In [2]:
def conndb(db_name):
    import pymysql
    host_name = 'localhost'
    host_port = 3306
    user_name = 'root'
    password = '1234'
    db = pymysql.connect(host=host_name, port=host_port, user=user_name, passwd=password,
                         db=db_name, charset='utf8')
    return db

In [20]:
db = conndb('beauty_shop')
cursor = db.cursor()

In [8]:
sql = "SHOW DATABASES;"
cursor.execute(sql)
result = cursor.fetchall()
result

(('beauty_shop',),
 ('ecommerce',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sqldb',),
 ('student_mgmt',),
 ('sys',),
 ('tabledb',),
 ('world',))

In [9]:
sql = "USE beauty_shop;"
cursor.execute(sql)

0

In [10]:
sql = "SELECT DATABASE()"
cursor.execute(sql)
result = cursor.fetchone()
result

('beauty_shop',)

In [11]:
# 테이블 생성
sql = """
    CREATE TABLE product (
        PRODUCT_CODE INT AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE VARCHAR(100),
        DISCOUNT_PRICE VARCHAR(100),
        link VARCHAR(200),
        CONSTRAINT PRIMARY KEY PK_product_PRODUCT_CODE (PRODUCT_CODE)
    );
"""

cursor.execute(sql)
db.commit()

In [12]:
sql = 'DESC product;'
cursor.execute(sql)
result = cursor.fetchall()
result

(('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

크롤링 대상: https://jolse.com/
해당 사이트에서 Toner & Mists 제품명, 가격, 세일가격, 링크 추출

In [21]:
url = 'https://jolse.com/category/toners-mists/1019/'
html = urlopen(url)
htmls = html.read()
bs_obj = BeautifulSoup(htmls, 'html.parser')

In [22]:
# box 안의 1개의 상품에서 정보를 추출, dict 형태로 반환
def get_product_info(box) :
    strong_tag = box.find("strong",{"class":"name"})
    # 품목 추출
    span =strong_tag.text.split(':')[1]
    
    # 세부페이지링크 추출
    a = strong_tag.find("a")
    sub_link = 'https://jolse.com' + a["href"]
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = "'"+price_ul.findAll("span")[1].text+"'"
    sals_price = "'"+box.find('ul').findAll('span')[-1].text+"'"
    
    # 데이터 전처리
    title = span.replace("'","''") # ' 처리
    ord_price = price_span
    dis_price = sals_price
   # ord_price = price_span[1].text.split(' ')[1] # USD 제거
   # dis_price = price_span[-1].text.split(' ')[1]
    # 세일 가격이 없는 경우 
    if dis_price =='' :
        dis_price = '0.0'
    
    
    # 최종 data 추출 후 반환 
    return{"prd_name":title,"price":ord_price,"sale_price":dis_price,"sub_link":sub_link}

In [23]:
def save_data(prd_info) :
    #     print(prd_info)
    
    # insert 구문
    sql = "INSERT INTO product (title, ori_price, discount_price,link) values('" \
        + prd_info["prd_name"] \
        +"'," \
        + prd_info['price']\
        +"," \
        + prd_info['sale_price']\
        +",'"\
        + prd_info['sub_link']\
        + "');"
    print(sql)
    cursor.execute(sql)

In [24]:
def get_page_products(url) :
    url=url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid5"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    for i in ul:
        prd_boxes = ul.findAll("div", {"class":"description"}) #1개 페이지의 전체 상품
        # 반환되는 품목 데이터를 db에 insert  : 함수호출해서 진행
        for box in prd_boxes :
            prd = get_product_info(box)
            print(prd)
            save_data(prd) #사용자 정의 함수(생성해야 함)

In [25]:
from tqdm import tqdm_notebook # 상태바 표시

#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열

#last= int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])
last=3
for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집
# for i in range(1,2) : # 1페이지 insert
    # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    get_page_products(urlfin)

C:\Users\hw981\AppData\Local\Temp\ipykernel_9700\3187220269.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집


  0%|          | 0/2 [00:00<?, ?it/s]

{'prd_name': ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml', 'price': "'USD 37.00'", 'sale_price': "'USD 33.30'", 'sub_link': 'https://jolse.com/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/'}
INSERT INTO product (title, ori_price, discount_price,link) values(' Haruharu WONDER Black Rice Hyaluronic Toner 300ml','USD 37.00','USD 33.30','https://jolse.com/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/');
{'prd_name': ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml', 'price': "'USD 22.00'", 'sale_price': "'USD 15.40'", 'sub_link': 'https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/'}
INSERT INTO product (title, ori_price, discount_price,link) values(' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml','USD 22.00','USD 15.40','https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1

In [26]:
db.commit()

In [27]:
sql = "SELECT * FROM product"
cursor.execute(sql)
result = cursor.fetchall()
result

((29,
  ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  'USD 37.00',
  'USD 33.30',
  'https://jolse.com/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/'),
 (30,
  ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml',
  'USD 22.00',
  'USD 15.40',
  'https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/'),
 (31,
  ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  'USD 37.00',
  'USD 33.30',
  'https://jolse.com/product/haruharu-wonder-black-rice-hyaluronic-toner-300ml/37796/category/1019/display/2/'),
 (32,
  ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml',
  'USD 22.00',
  'USD 15.40',
  'https://jolse.com/product/some-by-mi-propolis-b5-glow-barrier-calming-toner-150ml/42205/category/1019/display/2/'),
 (33,
  ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  'USD 37.00',
  'USD 33.30',
  'https://jolse.com/product/haruharu-wonder-black-rice-hyaluron

In [28]:
# DataFrame으로 가져오기
sql = 'SELECT * FROM product'
df = pd.read_sql(sql, db)
df

C:\Users\hw981\AppData\Local\Temp\ipykernel_9700\3628537493.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,29,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30,https://jolse.com/product/haruharu-wonder-blac...
1,30,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40,https://jolse.com/product/some-by-mi-propolis-...
2,31,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30,https://jolse.com/product/haruharu-wonder-blac...
3,32,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40,https://jolse.com/product/some-by-mi-propolis-...
4,33,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30,https://jolse.com/product/haruharu-wonder-blac...
5,34,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40,https://jolse.com/product/some-by-mi-propolis-...
6,35,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30,https://jolse.com/product/haruharu-wonder-blac...
7,36,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40,https://jolse.com/product/some-by-mi-propolis-...
8,37,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30,https://jolse.com/product/haruharu-wonder-blac...
9,38,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40,https://jolse.com/product/some-by-mi-propolis-...


In [29]:
sql = "SELECT TITLE, ORI_PRICE, DISCOUNT_PRICE FROM product"
df = pd.read_sql(sql, db)
df

C:\Users\hw981\AppData\Local\Temp\ipykernel_9700\1634852644.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,TITLE,ORI_PRICE,DISCOUNT_PRICE
0,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40
2,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
3,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40
4,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
5,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40
6,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
7,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40
8,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 33.30
9,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 15.40
